In [1]:
import numpy as np
import pickle
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential, model_from_json
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
def normalize_data(x):
    #print(x.shape)
    sq = np.multiply(x, x)
    mag = np.sqrt(np.sum(sq, axis = 2, keepdims=True))
    #print(mag.shape)    
    #mag = np.concatenate((mag, mag, mag), axis = 2)
    #print(mag.shape)
    x = np.divide(x, mag)
    return x

def normalize_data_all(data):
    for i in range(len(data)):        
        for subject in range(len(data[i])):
            for sess in range(len(data[i][subject])):
                x = data[i][subject][sess]["x"]
                data[i][subject][sess]["x"] = normalize_data(x)
    return data    
    

In [3]:
def get_train_data(data, exclude_subject):
    
    X_train, Y_train = [], []
    for i in range(4):        
        for subject in range(len(data[i])):
            if (i ==1 and subject == exclude_subject+2) or (i ==3 and subject == exclude_subject):
                continue
            for sess in range(len(data[i][subject])):
                x = data[i][subject][sess]["x"]
                y = data[i][subject][sess]["y"]
                
                if i<2:
                    if len(X_train)==0:
                        X_train = x
                        Y_train = y
                    else:
                        X_train = np.concatenate((X_train, x), axis = 0)
                        Y_train = np.concatenate((Y_train, y), axis = 0)
                        
                elif i==3:
                    x = x[y==0, :, :]
                    y = y[y==0]
                    
                    X_train = np.concatenate((X_train, x), axis = 0)
                    Y_train = np.concatenate((Y_train, y), axis = 0)                                    
    
    return X_train, Y_train

In [4]:
#x = X_train[5, :, :]
#x = np.sqrt(np.sum(np.multiply(x, x), axis = 1))
#print(x)

In [5]:
def train_model(X_train, Y_train, batch_size, epochs):
    input_shape = (X_train.shape[1], X_train.shape[2], 1)

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), padding ='valid', strides=(1, 1),
                     activation='relu',
                     input_shape=input_shape ))
    model.add(MaxPooling2D(pool_size=(4, 1), strides=(4, 1)))

    model.add(Conv2D(64, kernel_size=(2, 2), padding ='valid', strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 1), strides=(4, 1)))


    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=binary_crossentropy,
                  optimizer= Adam(),
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs)

    return model

In [6]:
def test_model(model, X_test, Y_test):
    Ypr = model.predict(X_test, verbose=0)
    Yprb = Ypr[:, 0]>0.5
    Yprb = Yprb.reshape((len(Y_test), 1))
    Y = np.concatenate((Y_test, Yprb, Ypr), axis=1)
    return Y

In [9]:
with open("C:\\ASM\\DevData\\eating\\python\\segments.pkl", "rb") as file:
    data = pickle.load(file)
data = normalize_data_all(data)
    
batch_size = 64
epochs = 50

res = []
for subject in range(6, 11):
    print("Subject: ", subject)
    X_train, Y_train = get_train_data(data, subject)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    Y_train = Y_train.reshape(Y_train.shape[0], 1)
    print("X_train shape: ", X_train.shape)
    print("Y_train shape: ", Y_train.shape)
    model = train_model(X_train, Y_train, batch_size, epochs)
    
    path = "C:\\ASM\\DevData\\eating\\python\\models\\"    
    model_json = model.to_json()
    with open(path+"model_"+str(subject)+".json", "w") as json_file:
        json_file.write(model_json)    
    model.save_weights(path+"model_"+str(subject)+".h5")
    print("Saved model to disk")
    
    print("Testing ...")
    res_lab = []
    if subject<=4:        
        path = "C:\\ASM\\DevData\\eating\\python\\results\\lab_"
        for sess in range(len(data[5][subject+2])):
            X_test = data[5][subject+2][sess]["x"]
            Y_test = data[5][subject+2][sess]["y"]
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
            Y_test = Y_test.reshape(Y_test.shape[0], 1)            
            Y = test_model(model, X_test, Y_test)
            res_lab.append(Y)
            np.savetxt(path+str(subject)+"_"+str(sess)+".csv", Y, delimiter=",")
        
    res_free = []
    path = "C:\\ASM\\DevData\\eating\\python\\results\\free_"
    for sess in range(len(data[3][subject])):
        X_test = data[3][subject][sess]["x"]
        Y_test = data[3][subject][sess]["y"]
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
        Y_test = Y_test.reshape(Y_test.shape[0], 1)        
        Y = test_model(model, X_test, Y_test)
        res_free.append(Y)
        np.savetxt(path+str(subject)+"_"+str(sess)+".csv", Y, delimiter=",")
        
    res_subject = {"lab":res_lab, "free":res_free}    
    res.append(res_subject)
    print(" Done")
    
path = "C:\\ASM\\DevData\\eating\\python\\results\\res.pkl"
#with open(path, "wb") as file:
#    pickle.dump(res, file)

Subject:  6
X_train shape:  (37779, 128, 3, 1)
Y_train shape:  (37779, 1)
Epoch 1/50
37779/37779 [==============================] - 5s 135us/step - loss: 0.2438 - acc: 0.9161
Epoch 2/50
37779/37779 [==============================] - 5s 121us/step - loss: 0.1967 - acc: 0.9302
Epoch 3/50
37779/37779 [==============================] - 5s 122us/step - loss: 0.1838 - acc: 0.9334
Epoch 4/50
37779/37779 [==============================] - 5s 121us/step - loss: 0.1729 - acc: 0.9372
Epoch 5/50
37779/37779 [==============================] - 5s 122us/step - loss: 0.1645 - acc: 0.9398
Epoch 6/50
37779/37779 [==============================] - 5s 126us/step - loss: 0.1571 - acc: 0.9428
Epoch 7/50
37779/37779 [==============================] - 5s 124us/step - loss: 0.1514 - acc: 0.9438
Epoch 8/50
37779/37779 [==============================] - 5s 125us/step - loss: 0.1456 - acc: 0.9472
Epoch 9/50
37779/37779 [==============================] - 5s 126us/step - loss: 0.1397 - acc: 0.9499
Epoch 10/50
37779

In [13]:
path = "C:\\ASM\\DevData\\eating\\python\\results\\"
res = []
for subject in range(11):
    res_lab = [];
    if subject<=4:
        sess_count = 2
        if subject<=1:
            sess_count = 1
        for sess in range(sess_count):
            res_sess = np.genfromtxt(path+"lab_"+str(subject)+"_"+str(sess)+".csv", delimiter=",")
            print(res_sess.shape)
            res_lab.append(res_sess)
    
    res_free = []
    sess_count = 2
    if subject==9:
        sess_count = 5
    for sess in range(sess_count):
        res_sess = np.genfromtxt(path+"free_"+str(subject)+"_"+str(sess)+".csv", delimiter=",")
        res_free.append(res_sess)
        
    res_subject = {"lab":res_lab, "free":res_free}    
    res.append(res_subject)
    
path = "C:\\ASM\\DevData\\eating\\python\\results\\res.pkl"
with open(path, "wb") as file:
    pickle.dump(res, file)

(933, 3)
(652, 3)
(571, 3)
(384, 3)
(457, 3)
(503, 3)
(731, 3)
(601, 3)
